In [15]:
import pandas as pd
import os
import datetime as dt
from helper import *
from paths import *

In [12]:
hj_nr, prev_start, prev_end = get_half_year_info().values()
last_HJ = f'Z{hj_nr}'

In [ ]:
kunden_segments = pd.read_excel(ks_path)
kunden_segment_dict = dict(zip(kunden_segments['Alt'],kunden_segments['Neu']))

kw = pd.read_csv(kw_path, sep=';',encoding='cp850',on_bad_lines='skip')
adresse = pd.read_csv(adresse_path, sep=';',encoding='cp850')
stat = pd.read_csv(stat_path, sep=';',encoding='cp850',usecols=['NUMMER','ERSTKAUF'])

In [ ]:
kw[last_HJ] = kw[last_HJ].map(kunden_segment_dict)

In [ ]:

kw['NUMMER'] = process_id(kw['NUMMER'])
adresse['NUMMER'] = process_id(adresse['NUMMER'])
stat['NUMMER'] = process_id(stat['NUMMER'])
adresse['GEBURT'] = process_date(adresse['GEBURT'])
adresse = assign_age(adresse)

In [ ]:
df = pd.merge(adresse,kw[['NUMMER',last_HJ]],on='NUMMER',how='left')
df = pd.merge(df,stat,on='NUMMER',how='left')

In [113]:
df = df[['NUMMER', 'ANREDE', 'TITEL', 'VORNAME', 'NAME','QUELLE', 'GEBURT', 'LKZ', 'PLZ', 'ORT','SYS_ANLAGE', 'SYS_BEWEG','ERSTKAUF', last_HJ]]

In [ ]:
df.loc[(df['ERSTKAUF']>=prev_start)&(df['ERSTKAUF']<prev_end),last_HJ] = "Neukunden-1"
df.loc[(df['ERSTKAUF'].isna()),last_HJ] = "Interessenten"


In [ ]:
df = df[df['SYS_ANLAGE']<prev_end]

In [120]:
df.to_csv('kw.csv',sep=';',encoding='cp850',index=False)